# Reduction package for 1.3m DFOT

Ccdproc is an Astropy affiliated package for basic data reductions of CCD images. It provides the essential tools for the cleaning of CCD images.

# Steps to follow:
1. Slicing the images taken in kinetic mode to single frames.
2. Updating the Julian Dates (JD) for each sliced image to the header for time information.
3. Alignment of the images using ASTROALIGN
4. Cleaning the images
          
     a) Bias correction.
     b) Flat correction.
     c) Cosmic Ray removal


Following packages are necessary to import:

In [5]:
import numpy as np
import matplotlib as mp
import ccdproc,os,sys,time,random
from astropy import units as u
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.time import Time
from glob import glob
#from astroquery.astrometry_net import AstrometryNet
from astropy.wcs import WCS
import astroalign as aa
from astropy.nddata import CCDData
from astropy.stats import sigma_clipped_stats, SigmaClip
from astropy.visualization import ImageNormalize, LogStretch
from matplotlib.ticker import LogLocator
from astropy.stats import SigmaClip, mad_std

# 1) 2) Slicing the images from datacube & Updating the J.D. for each sliced image using header information 

Here, we are slicing the images from the datacube (i.e. that are taken in kinetic mode) and 
updating their time in JD (Julian Date) by taking information from the header.

    INPUT:

    path: The directory where the images are kept (string)
    filename: The first few characters and the extension of the images (string). Example:
    j0946*fits, HD1104*fits etc.


    OUTPUT:

    1. The images sliced in the case of kinetic mode images.
    2. The Julian date updated to the header of the fits files.

In [11]:
path='/home/vivek/phot_dfot/dimple/photometry/data/'
filename='j0947_i*fits'

files=sorted(glob(os.path.join(path,filename)))
nof=np.zeros(len(files))
for i in range(0,len(files)):
    data=fits.open(files[i])
    header=data[0].header
    image=data[0].data
    k=np.shape(image)
    nof[i]=k[0]

    check_header=header['ACQMODE']

    if (check_header=='Single Scan'):
        jd_up=image
        time=header['DATE']
        t=Time(time,format='isot',scale='utc')
        time_jd=t.jd
        header.insert(15,('JD',time_jd))
        files[i]
        mod_file_1=files[i].replace('.fits','')
        fits.writeto(mod_file_1+'_sliced'+'.fits',jd_up,header,overwrite=True)

        #print(files[i],t.jd,t.mjd,'single scan image')



    elif (check_header=='Kinetics'):

        print('kinetic mode image with no. of files:',nof[i])

        name_of_file=files[i]
        mod_file=name_of_file.replace('.fits','')
        time=header['DATE']
        t=Time(time,format='isot',scale='utc')
        t_jd=t.jd
        temp=int(nof[i])
        mod_jd=np.zeros(temp)
        exp_time=header['ACT']/86400  # for the 'day' from seconds calculation.
        mod_jd[0]=t_jd   # first frame time from the header and the next ones from the 'ACT'
        for j in range(1,temp):
            mod_jd[j]=mod_jd[j-1]+exp_time


        for k in range(0,len(mod_jd)):
            mod_header=header
            sliced_image=image[k]
            time_jd=mod_jd[k]
            mod_header.insert(15,('JD',time_jd))
            fits.writeto(mod_file+'_sliced_%g'%k+'.fits',sliced_image,mod_header,overwrite=True)
            print(mod_file+'_sliced_%g'%k+'.fits has been written')
            continue


    #os.system("mv *sliced* cleaned/")


[0. 0. 0. 0. 0. 0. 0. 0. 0.]
kinetic mode image with no. of files: 4.0
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_14_17_sliced_0.fits has been written
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_14_17_sliced_1.fits has been written
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_14_17_sliced_2.fits has been written
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_14_17_sliced_3.fits has been written


kinetic mode image with no. of files: 4.0
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_19_22_sliced_0.fits has been written
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_19_22_sliced_1.fits has been written
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_19_22_sliced_2.fits has been written
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_19_22_sliced_3.fits has been written
kinetic mode image with no. of files: 3.0
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_23_25_sliced_0.fits has been written
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_23_25_sliced_1.fits has been written
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_23_25_sliced_2.fits has been written
kinetic mode image with no. of files: 5.0
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_2_6_sliced_0.fits has been written
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_2_6_sliced_1.fits has been written
/home/vivek/phot_dfot/dimple/photometry/data/j0947_i_2_6_sliced_2.fi

# 3. Aligning the images

Astro-Align package is used to alighn the images


INPUT:

    path: The directory where the images are kept (string)
    filename: The first few characters and the extension of the images (string). Example:
    j0946*fits, HD1104*fits etc.
    ref_image: Reference image which will be used to align the images.

OUTPUT:

    aligned images.


In [14]:
filename="*sliced*fits"
ref_image="j0947_i_1_sliced.fits"
nfiles=sorted(glob(path+filename))
image_data=fits.open(path+ref_image)
reference_image=image_data[0].data
for i in range(len(nfiles)):
    image_data=fits.open(nfiles[i])
    source_image=image_data[0].data
    header=image_data[0].header
    image_aligned,footprint=aa.register(source_image,reference_image)

    aligned_file=nfiles[i].replace('.fits','')
    fits.writeto(aligned_file+'_aligned'+'.fits',image_aligned,header,overwrite=True)

    print('No. %i alignment done'%i)


No. 0 done
No. 1 done
No. 2 done
No. 3 done
No. 4 done
No. 5 done
No. 6 done
No. 7 done
No. 8 done
No. 9 done
No. 10 done
No. 11 done
No. 12 done
No. 13 done
No. 14 done
No. 15 done
No. 16 done
No. 17 done
No. 18 done
No. 19 done
No. 20 done
No. 21 done
No. 22 done
No. 23 done
No. 24 done


# Cleaning the images

This module is meant for cleaning the images. The tasks to be included are: bias correction, flat correction, trimming, overscan as well as the cosmic ray removal from the science cases. (For the time we are skipping the overscan and trimming part.)

INPUT:

    path: The directory where the images are kept (string)
    filename: The first few characters and the extension of the images (string). Example:
    j0946*fits, HD1104*fits etc.

OUTPUT:


    cleaned images in the new directory: path/cleaned

# a) BIAS correction.

In [21]:
gain = 2 * u.electron / u.adu  # gain and readout noise are properties of the CCD and will change for different CCDs.
readnoise = 7.5 * u.electron

#ra=input('Enter the RA of the source:   ')
#dec=input('Enter the DEC of the source: ')


bias_files = sorted(glob(os.path.join(path,'bias*.fits')))
biaslist = []
for i in range (0,len(bias_files)):
    data= ccdproc.CCDData.read(bias_files[i],unit='adu')
        #data = ccdproc.create_deviation(data, gain=gain, readnoise=readnoise)
        #data= data-(data.uncertainty.array)
    biaslist.append(data)
    
masterbias = ccdproc.combine(biaslist,method='average',sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std)
masterbias.write('masterbias.fits', overwrite=True)
mbias=ccdproc.CCDData.read('masterbias.fits',unit='adu')
print('Master bias generated')
print(" Mean and median of the masterbias: ",np.mean(masterbias), np.median(masterbias))




INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Master bias generated
 Mean and median of the masterbias:  515.4987641731899 515.5


# b) Flat correction.

In [22]:
flat_files=sorted(glob(os.path.join(dir,'flat*.fits')))
flatlist = []
for j in range(0,len(flat_files)):
    flat=ccdproc.CCDData.read(flat_files[j],unit='adu')
    flat_bias_removed=ccdproc.subtract_bias(flat,masterbias)
    flatlist.append(flat_bias_removed)

    def inv_median(a):
        return 1 / np.median(a)

masterflat = ccdproc.combine(flatlist,method='median', scale=inv_median,
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std)
masterflat.write('masterflat.fits', overwrite=True)
mflat=ccdproc.CCDData.read('masterflat.fits',unit='adu')
print('Master flat generated')
print(" Mean and median of the masterflat: ",np.mean(masterflat), np.median(masterflat))



INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Master flat generated
 Mean and median of the masterflat:  0.9992749321566833 0.9999320144129445


# c) Cosmic Ray removal.

In [ ]:
filename="*aligned*fits"
file_names = sorted(glob(os.path.join(dir,filename)))
for i in range(0,len(file_names)):
    image=ccdproc.CCDData.read(file_names[i],unit='adu')
    header=fits.getheader(file_names[i],0)
    bias_subtracted = ccdproc.subtract_bias(image, masterbias)
    flat_corrected = ccdproc.flat_correct(bias_subtracted, masterflat)
    cr_cleaned = ccdproc.cosmicray_lacosmic(flat_corrected,readnoise=7.5, sigclip=5,satlevel=65535,niter=20,cleantype='meanmask',gain_apply=True)
        #print('Cosmic rays removed')
    clean_file=file_names[i].replace('.fits','')
    fits.writeto(clean_file+'_cleaned.fits',cr_cleaned,header,overwrite=True)
    print('Image no-%i has been cleaned'%i)